In [36]:
import numpy as np
import math
import pandas as pd

In [37]:
datos = np.array([["audi", "gas", "std", 1],
                 ["audi", "gas", "std", 1],
                 ["audi", "gas", "std", 0],
                 ["audi", "gas", "std", 2],
                 ["mazda", "gas", "std", 1],
                 ["mazda", "gas", "std", 1],
                 ["mazda", "gas", "std", 2],
                 ["peugot", "diesel", "rwd", 2],
                 ["peugot", "diesel", "turbo", 2],
                 ["volvo", "gas", "turbo", 0],
                 ["volvo", "gas", "std", 1],
                 ["volvo", "gas", "turbo", 0]])

new_data = np.array([
    ["1", "F", 1],
    ["2", "M", 1],
    ["3", "F", 0],
    ["3", "M", 0],
    ["3", "F", 0],
    ["3", "F", 1],
    ["2", "F", 0],
    ["1", "F", 1],
    ["2", "M", 1],
    ["3", "F", 1]
])
                     
                     
                     
                     
                     
                     
                     
                     
                     
                     
                     

fisher_length = 0.3336
fisher_width = 0.2892
fisher_height = 0.4047
fisher_highway_mpg = 0.1335

In [38]:
#Calcular correlación
#Pij = 
print(new_data[:,0])

['1' '2' '3' '3' '3' '3' '2' '1' '2' '3']


In [55]:
def return_unique_characteristics(column: int, datos: list) -> list:
    characteristics = datos[:,column].tolist()
    characteristics

    unique_characteristics = []

    for characteristic in characteristics:
        if not(characteristic in unique_characteristics):
            unique_characteristics.append(characteristic)
    return unique_characteristics

def create_dictionary(unique_characteristics: list, total_class: int) -> dict:
    dictionary = dict()

    for characteristic in unique_characteristics:    
        for i in np.arange(total_class):
            key = str(characteristic) + str(i)
            dictionary.setdefault(key, 0)
    return dictionary

def gini(total_class: int, column: int, column_class: int, datos: list):
    unique_characteristics = return_unique_characteristics(column=column)
    dictionary = create_dictionary(unique_characteristics=unique_characteristics, total_class=total_class)

    for i in np.arange(datos.shape[0]):
        key = datos[i,column] + datos[i, column_class]
        dictionary[key] = dictionary[key] + 1


    total_sum = 0
    for characteristic in unique_characteristics:
        simple_sum = 0
        ni = 0
        for i in np.arange(total_class):
            key = str(characteristic) + str(i)
            simple_sum += dictionary[key] / datos.shape[0]
            ni += dictionary[key]
        simple_sum = 1 - simple_sum
        total_sum += (simple_sum * ni) / datos.shape[0] 
    print(total_sum)

def entropia(total_class: int, column: int, column_class: int, datos: list):
    unique_characteristics = return_unique_characteristics(column=column, datos=datos)
    dictionary = create_dictionary(unique_characteristics=unique_characteristics, total_class=total_class)

    for i in np.arange(datos.shape[0]):
        key = str(datos[i,column]) + str(datos[i, column_class])
        dictionary[key] = dictionary[key] + 1


    total_sum = 0
    for characteristic in unique_characteristics:
        simple_sum = 0
        ni = 0
        for i in np.arange(total_class):
            key = str(characteristic) + str(i)
            res = (dictionary[key] / datos.shape[0])
            if(res):
                res = res * math.log2(dictionary[key] / datos.shape[0])
            simple_sum += res
            ni += dictionary[key]
        simple_sum = - simple_sum
        total_sum += (simple_sum * ni) / datos.shape[0] 
    return total_sum
    #print(total_sum)

In [40]:
total_class = 3
column_class = 2
entropia(total_class=total_class, column=0, column_class=column_class, datos=new_data)
entropia(total_class=total_class, column=1, column_class=column_class, datos=new_data)


0.9738761698832273

# **Twitter**

In [41]:
df = pd.read_csv('Twitter.csv')

In [42]:
df.head()

,Unnamed: 0,nombre,cantidad_caracteres_nombre,tiene_foto_perfil,cantidad_seguidores,cantidad_seguidos,promedio_publicaciones,perfil_privado,dia_mas_publica,comenta_publicaciones,clase
0,0,Catalin31937081,largo,Sí,bajo,bajo,bajo,No,Sábado,Sí,Fake
1,1,JadeCrespo8,medio,Sí,bajo,medio,bajo,No,Martes,Sí,Fake
2,2,V12Ara,corto,Sí,bajo,bajo,bajo,No,Martes,Sí,Fake
3,3,BriannaLun_,medio,Sí,bajo,bajo,bajo,No,Martes,Sí,Fake
4,4,IsoldaMancillas,largo,Sí,bajo,bajo,bajo,No,Martes,Sí,Fake


In [43]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('^nombre')]

In [44]:
df['clase'] = df['clase'].replace({'Fake': 0, 'Real': 1})
df['clase'] = df['clase'].replace({'FALSO': 0, 'real': 1})

In [45]:
df.tail()

,cantidad_caracteres_nombre,tiene_foto_perfil,cantidad_seguidores,cantidad_seguidos,promedio_publicaciones,perfil_privado,dia_mas_publica,comenta_publicaciones,clase
289,medio,si,bajo,bajo,bajo,no,sabado,no,0
290,medio,si,bajo,bajo,bajo,no,domingo,no,0
291,largo,si,bajo,bajo,bajo,no,miercoles,no,0
292,largo,si,medio,alto,bajo,no,sabado,no,0
293,medio,si,medio,medio,bajo,no,miercoles,no,0


In [46]:
twitter = df.to_numpy()

In [47]:
entropis = []
for i in np.arange(8):
    total_class = 2
    column_class = 8
    entropis.append(entropia(total_class=total_class, column=i, column_class=column_class, datos=twitter))
entropis = np.array(entropis)
print(entropis)

[0.85632273 0.83023047 0.7598031  0.78977323 0.8717221  0.82573526
 0.33491488 0.72101021]


In [48]:
print(entropis.min())

0.33491488013992987


# **Facebook**

In [56]:
df = pd.read_csv('Facebook.csv')

In [57]:
df.head()

,Unnamed: 0,caracteres_nombre,foto_perfil,num_amigos,dias_mas_Publicaciones,paginas_que_sigue,Clase
0,0,corto,Sí,medio,Domingo,pocas,T
1,1,medio,Sí,medio,Domingo,pocas,T
2,2,medio,Sí,medio,Domingo,pocas,T
3,3,corto,Sí,medio,Domingo,pocas,T
4,4,corto,Sí,medio,Viernes,pocas,T


In [58]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Clase'] = df['Clase'].replace({'T': 0, 'F': 1})

In [59]:
df.tail()

,caracteres_nombre,foto_perfil,num_amigos,dias_mas_Publicaciones,paginas_que_sigue,Clase
356,largo,SI,medio,Sabado,algunas,0
357,medio,SI,bajo,ninguno,pocas,1
358,corto,SI,bajo,ninguno,pocas,1
359,medio,SI,alto,ninguno,pocas,1
360,largo,SI,medio,ninguno,pocas,0


In [60]:
facebook = df.to_numpy()

In [62]:
entropis = []
for i in np.arange(4):
    total_class = 2
    column_class = 5
    entropis.append(entropia(total_class=total_class, column=i, column_class=column_class, datos=facebook))
entropis = np.array(entropis)
print(entropis)

{'corto0': 0, 'corto1': 0, 'medio0': 0, 'medio1': 0, 'largo0': 0, 'largo1': 0}
corto0
medio0
medio0
corto0
corto0
largo0
medio0
medio0
medio0
medio0
largo0
medio0
medio0
largo0
medio0
medio0
corto0
medio0
corto0
medio0
medio0
corto0
largo0
medio0
medio0
medio0
medio0
largo0
medio0
medio0
medio0
medio0
corto0
medio0
largo0
medio0
corto0
medio0
medio0
medio0
largo0
medio0
medio0
medio0
medio0
medio0
medio0
medio0
medio0
corto0
medio0
medio0
largo0
medio0
medio0
medio0
medio0
medio0
largo1
medio1
medio1
largo1
medio1
medio1
largo1
largo1
medio1
medio0
medio0
medio0
largo0
largo0
corto0
corto0
largo0
medio0
medio0
medio0
medio0
medio0
medio0
medio0
medio1
medio1
medio1
medio0
medio0
largo0
largo0
medio0
medio0
largo0
medio0
medio0
medio0
medio0
medio0
medio0
medio0
medio1
medio1
medio1
medio1
medio1
medio1
medio1
medio1
medio1
medio0
medio0
medio1
medio0
corto0
medio0
corto1
medio0
medio0
medio0
medio0
medio0
corto0
medio0
medio0
corto0
medio0
medio0
corto0
medio0
medio0
medio0
medio0
cort